In [ ]:
from sys import exit
from flask import Flask
from argparse import ArgumentParser
from configs import settings, configurations
from utils.filehandler import FilePreprocess
from CRD.views import CreateData, ReadData, DeleteData

parser = ArgumentParser()
parser.add_argument('--datastore', help='Enter the datastore absolute path.')
args = parser.parse_args()
if args.datastore:
    db_path = args.datastore
else:
    db_path = configurations.DEFAULT_DB_PATH
directory_created = FilePreprocess(db_path).create_folder()
if not directory_created:
    print(f"Permission denied: You can not create the directory `{db_path}`.\n")
    exit(0)
app = Flask(__name__)
app.config['DEBUG'] = settings.DEBUG
app.config['SECRET_KEY'] = settings.SECRET_KEY
app.add_url_rule('/datastore/create', view_func=CreateData.as_view('create', db_path), methods=['POST'])
app.add_url_rule('/datastore/read', view_func=ReadData.as_view('read', db_path), methods=['GET'])
app.add_url_rule('/datastore/delete', view_func=DeleteData.as_view('delete', db_path), methods=['DELETE'])
if __name__ == '__main__':
    app.run(host=settings.HOST, port=settings.PORT)


In [ ]:
#CREATING THE DATA
from sys import exit
from utils.filehandler import FilePreprocess
from CRD.functions import DataStoreCRD
from argparse import ArgumentParser
from configs import configurations

parser = ArgumentParser()
parser.add_argument('--datastore', help='Enter the datastore absolute path.')
args = parser.parse_args()
if args.datastore:
    db_path = args.datastore
else:
    db_path = configurations.DEFAULT_DB_PATH
directory_created = FilePreprocess(db_path).create_folder()
if not directory_created:
    print(f"Permission denied: You can not create the directory `{db_path}`.\n")
    exit(0)
json_data = {
    "abc": {
        "data1": "value1",
        "data2": "value2",
        "data3": "value3",
        "Time-To-Live": 5000,
    },
    "def": {
        "data1": "value1",
        "data2": "value2",
        "data3": "value3",
        "Time-To-Live": 50,
    },
    "ghi": {
        "data1": "value1",
        "data2": "value2",
        "data3": "value3",
        "data4": "value4",
    },
    "jkl": {
        "data1": "value1",
        "data2": "value2",
        "data3": "value3",
        "Time-To-Live": 250,
    }
}
''' Creating data '''
_valid_data, message = DataStoreCRD().check_create_data(json_data, db_path)
print(message)



In [ ]:
#READING THE DATA
from sys import exit
from argparse import ArgumentParser
from configs import configurations
from utils.filehandler import FilePreprocess
from CRD.functions import DataStoreCRD


parser = ArgumentParser()
parser.add_argument('--datastore', help='Enter the datastore absolute path.')
args = parser.parse_args()
if args.datastore:
    db_path = args.datastore
else:
    db_path = configurations.DEFAULT_DB_PATH
directory_created = FilePreprocess(db_path).create_folder()
if not directory_created:
    print(f"Permission denied: You can not create the directory `{db_path}`.\n")
    exit(0)
key = 'ghi'
''' Reading data '''
_data_found, message = DataStoreCRD().check_read_data(key, db_path)
print(message)



In [ ]:
#DELETING THE DATA
from sys import exit
from argparse import ArgumentParser
from configs import settings, configurations
from utils.filehandler import FilePreprocess
from CRD.functions import DataStoreCRD

parser = ArgumentParser()
parser.add_argument('--datastore', help='Enter the datastore absolute path.')
args = parser.parse_args()
if args.datastore:
    db_path = args.datastore
else:
    db_path = configurations.DEFAULT_DB_PATH
directory_created = FilePreprocess(db_path).create_folder()
if not directory_created:
    print(f"Permission denied: You can not create the directory `{db_path}`.\n")
    exit(0)
key = 'ghi'
'''deleting data '''
_data_found, message = DataStoreCRD().check_delete_data(key, db_path)
print(message)



In [ ]:
#FILE HANDLING
from os import path, makedirs
class FilePreprocess:
    def __init__(self, file_path):
        self.file_path = file_path

    def create_folder(self):
        try:
            makedirs(self.file_path, mode=0o777, exist_ok=True)
        except PermissionError:
            return False
        return True


In [ ]:
#REQUIRED FUNCTIONS
import json
import fcntl
import threading
from os import path
from datetime import datetime, timedelta
from dateutil.parser import parse
from configs.configurations import DEFAULT_DB_NAME


class DataStoreCRD:
    def check_time_to_live(self, value):
        created_time = value['CreatedAt']
        created_time = parse(created_time)
        time_to_live = value['Time-To-Live']
        if time_to_live is not None:
            expired_datetime = created_time + timedelta(seconds=time_to_live)
            remaining_seconds = (expired_datetime - datetime.now()).total_seconds()
        if remaining_seconds <= 0:
                return False
        return value

    def check_create_data(self, json_data, db_path):
        if not isinstance(json_data, dict):
            return False, "Incorrect request data format. Only JSON object with key-value pair is acceptable."
        data_obj = json.dumps(json_data)
        if len(data_obj) > 1000000000:
            return False, "DataStore limit will exceed 1GB size."
        for key, value in json_data.items():
            if len(key) > 32:
                return False, "The keys must be in 32 characters length."
            if not isinstance(value, dict):
                return False, "The values must be in JSON object formats."
            value_obj = json.dumps(value)
            if len(value_obj) > 16384:
                return False, "The values must be in 16KB size."
        datastore = path.join(db_path, DEFAULT_DB_NAME)
        data = {}
        if path.isfile(datastore):
            with open(datastore) as f:
                fcntl.flock(f, fcntl.LOCK_EX)
                data = json.load(f)
                fcntl.flock(f, fcntl.LOCK_UN)
                prev_data_obj = json.dumps(data)
                if len(prev_data_obj) >= 1000000000:
                    return False, "File Size Exceeded 1GB."
        have_key = any(x in json_data.keys() for x in data.keys())
        if have_key:
            return False, "Key already exist in DataStore."
        def prepare_data_create(json_data_keys):
            for key in json_data_keys:
                singleton_json = json_data[key]
                singleton_json["CreatedAt"] = datetime.now().isoformat()
                singleton_json["Time-To-Live"] = singleton_json["Time-To-Live"] if 'Time-To-Live' in singleton_json else None
                data[key] = singleton_json
        thread_count = 3
        items = list(json_data.keys())
        split_size = len(items) // thread_count
        threads = []
        for i in range(thread_count):
            start = i * split_size
            end = None if i+1 == thread_count else (i+1) * split_size
            threads.append(threading.Thread(target=prepare_data_create, args=(items[start:end], ), name=f"t{i+1}"))
            threads[-1].start()
        for t in threads:
            t.join()
        with open(datastore, 'w+') as f:
            fcntl.flock(f, fcntl.LOCK_EX)
            json.dump(data, f)
            fcntl.flock(f, fcntl.LOCK_UN)
        return True, "Data created in DataStore."
    def read_delete_preprocess(self, key, db_path):
        datastore = path.join(db_path, DEFAULT_DB_NAME)
        if not path.isfile(datastore):
            return False, "Empty DataStore. Data not found for the key."
        with open(datastore) as f:
            fcntl.flock(f, fcntl.LOCK_EX)
            data = json.load(f)
            fcntl.flock(f, fcntl.LOCK_UN)
        if key not in data.keys():
            return False, "No data found for the key provided."
        target = data[key]
        target_active = self.check_time_to_live(target)
        if not target_active:
            return False, "Requested data is expired for the key."
        return True, data
    def check_read_data(self, key, db_path):
        status, message = self.read_delete_preprocess(key, db_path)
        if not status:
            return status, message
        data = message[key]
        del data['CreatedAt']
        return status, data
    def check_delete_data(self, key, db_path):
        status, message = self.read_delete_preprocess(key, db_path)
        if not status:
            return status, message
        datastore = path.join(db_path, DEFAULT_DB_NAME)
        del message[key]
        with open(datastore, 'w+') as f:
            fcntl.flock(f, fcntl.LOCK_EX)
            json.dump(message, f)
            fcntl.flock(f, fcntl.LOCK_UN)
        return True, "Data is deleted from the datastore."


In [ ]:
#VIEWS
from flask.views import MethodView
from flask import jsonify, request
from CRD.functions import DataStoreCRD


class CreateData(MethodView):
    def __init__(self, db_path):
        self.db_path = db_path

    def post(self):
        try:
            json_data = request.get_json(force=True)
        except Exception:
            return jsonify({"status": "error", "message": "Incorrect request data format. Only JSON object is acceptable."}), 400

        valid_data, message = DataStoreCRD().check_create_data(json_data, self.db_path)
        if not valid_data:
            return jsonify({"status": "error", "message": message}), 400

        return jsonify({"status": "success", "message": message}), 200


class ReadData(MethodView):
    def __init__(self, db_path):
        self.db_path = db_path

    def get(self):
        key = request.args.get('key')
        if key is None:
            return jsonify({"status": "error", "message": "key is required as a query param."}), 400

        data_found, message = DataStoreCRD().check_read_data(key, self.db_path)
        if not data_found:
            return jsonify({"status": "error", "message": message}), 404

        return jsonify(message), 200


class DeleteData(MethodView):
    def __init__(self, db_path):
        self.db_path = db_path

    def delete(self):
        key = request.args.get('key')

        if key is None:
            return jsonify({"status": "error", "message": "key is required as a query param."}), 400

        data_found, message = DataStoreCRD().check_delete_data(key, self.db_path)
        if not data_found:
            return jsonify({"status": "error", "message": message}), 404

        return jsonify({"status": "success", "message": message}), 200
